In [3]:
import os
import pandas as pd
import numpy as np
pd.set_option("display.max_colwidth", 2)

os.chdir("/home/coinse/Workspace/SBFL")

# Load bug info

In [4]:
from scripts.util import load_bug_data, find_covmat_func, find_covmat_idx
from functools import reduce
from typing import Set

pid, vid = "libchewing", 7

bug = load_bug_data(pid, vid)

print(f"{bug.buggy_files=}")
print(f"{bug.buggy_lines=}")
print("bug.coverage_df=")
display(bug.coverage_df)
print(f"{bug.failing_tests=}")
print(f"{bug.failing_tests_GT=}")
print("bug.functions={", end="")
for func in list(bug.functions)[:5]:
    print(f"{func},", end=" ")
print(", ...}")
print(f"{bug.id=}")
print(f"{bug.num_test_cases=}")
print(f"{bug.passing_tests=}")
print(f"{bug.path_to_cov_df=}")
print(f"{bug.path_to_patch=}")
print(f"{bug.path_to_test_results=}")
print("bug.source_files=", end="")
for src in list(bug.source_files)[:5]:
    print(f"{src},", end=" ")
print(", ...}")

fault_idxs = sorted(bug.buggy_lines)
# fault_idx2meth = {}
fault_meth2idx = {}
err_msg = ""
for fault_idx in fault_idxs:
    succ_func, ret_func = find_covmat_func(
        pid, bug.coverage_df, fault_idx, debug=False
    )
    if succ_func:
        succ_idx, ret_idx = find_covmat_idx(
            pid, bug.coverage_df, fault_idx, debug=False
        )
        if succ_idx:
            if ret_func in fault_meth2idx:
                fault_meth2idx[ret_func].add(ret_idx)
            else:
                fault_meth2idx[ret_func] = {ret_idx}
        else:
            print(f"{ret_idx}")
            if ret_func not in fault_meth2idx:
                fault_meth2idx[ret_func] = set()
    else:
        print(f"{ret_func}")
# fault_meths: Set = reduce(
#     lambda x, y: x | set([y]), fault_idx2meth.values(), set()
# )
# fault_meth2idx = {
#     v: {idx for idx in fault_idx2meth if fault_idx2meth[idx] == v}
#     for v in fault_meths
# }
fault_meths = set(fault_meth2idx.keys())
for meth in fault_meth2idx:
    print(f"{meth=}: {fault_meth2idx[meth]}")
# print(f"{fault_meths=}")


bug.buggy_files={'src/chewingutil.c'}
bug.buggy_lines={('src/chewingutil.c', 731), ('src/chewingutil.c', 732)}
bug.coverage_df=


16  \
file                                              function           line         
/home/workspace/src/common/chewing-utf8-util.c    ueStrLen           28    740    
                                                                     30    740    
                                                                     31    740    
                                                                     33    2321   
                                                                     34    1581   
...                                                                         ...   
/home/workspace/src/porting_layer/src/plat_path.c find_path_by_files 191   16     
                                                                     192   16     
                                                                     193   16     
                                                                     194   16     
                                                                     197   0      

                                                                             3  \
file                                              function           line        
/home/workspace/src/common/chewing-utf8-util.c    ueStrLen           28    145   
                                                                     30    145   
                                                                     31    145   
                                                                     33    896   
                                                                     34    751   
...                                                                        ...   
/home/workspace/src/porting_layer/src/plat_path.c find_path_by_files 191   8     
                                                                     192   8     
                                                                     193   8     
                                                                     194   8     
                                                                     197   0     

                                                                             6  \
file                                              function           line        
/home/workspace/src/common/chewing-utf8-util.c    ueStrLen           28    13    
                                                                     30    13    
                                                                     31    13    
                                                                     33    119   
                                                                     34    106   
...                                                                        ...   
/home/workspace/src/porting_layer/src/plat_path.c find_path_by_files 191   0     
                                                                     192   0     
                                                                     193   0     
                                                                     194   0     
                                                                     197   0     

                                                                            13  \
file                                              function           line        
/home/workspace/src/common/chewing-utf8-util.c    ueStrLen           28    46    
                                                                     30    46    
                                                                     31    46    
                                                                     33    393   
                                                                     34    347   
...                                                                        ...   
/home/workspace/src/porting_layer/src/plat_path.c find_path_by_files 191   4     
                                                                     192   4     
                                           

bug.failing_tests={'1'}
bug.failing_tests_GT={'1'}
bug.functions={('/home/workspace/src/tree.c', 'SortByIncreaseEnd'), ('/home/workspace/src/bopomofo.c', 'BopomofoRemoveLast'), ('/home/workspace/src/chewingutil.c', 'SpecialSymbolInput'), ('/home/workspace/src/common/chewing-utf8-util.c', 'ueBytesFromChar'), ('/home/workspace/src/userphrase-sql.c', 'UserBindPhone'), , ...}
bug.id='libchewing-7'
bug.num_test_cases=18
bug.passing_tests={'6', '7', '3', '8', '13', '5', '17', '9', '10', '2', '16', '18', '14', '4', '15', '11', '12'}
bug.path_to_cov_df='/home/coinse/Workspace/defects4cpp/coverage/libchewing_7_buggy/coverage_df.pkl'
bug.path_to_patch='/home/coinse/Workspace/defects4cpp/defects4cpp/taxonomy/libchewing/patch/0007-buggy.patch'
bug.path_to_test_results='/home/coinse/Workspace/defects4cpp/coverage/libchewing_7_buggy/test_results.json'
bug.source_files=/home/workspace/src/common/chewing-utf8-util.c, /home/workspace/src/chewingio.c, /home/workspace/src/porting_layer/src/plat_path.c, /

# Get spectra

In [5]:
covmat = bug.coverage_df
pass_tcs = bug.passing_tests
fail_tcs = bug.failing_tests

covmat_bool = covmat.astype(bool)
P = len(pass_tcs)
F = len(fail_tcs)
e_p = covmat_bool[pass_tcs].sum(axis=1)
e_f = covmat_bool[fail_tcs].sum(axis=1)
n_p = P - e_p
n_f = F - e_f
spectra = pd.DataFrame(
    {
        "line": covmat.index.to_list(),
        "e_p": e_p,
        "e_f": e_f,
        "n_p": n_p,
        "n_f": n_f,
    }
)

display(spectra)


line  \
file                                              function           line                                                                                 
/home/workspace/src/common/chewing-utf8-util.c    ueStrLen           28    (/home/workspace/src/common/chewing-utf8-util.c, ueStrLen, 28)                 
                                                                     30    (/home/workspace/src/common/chewing-utf8-util.c, ueStrLen, 30)                 
                                                                     31    (/home/workspace/src/common/chewing-utf8-util.c, ueStrLen, 31)                 
                                                                     33    (/home/workspace/src/common/chewing-utf8-util.c, ueStrLen, 33)                 
                                                                     34    (/home/workspace/src/common/chewing-utf8-util.c, ueStrLen, 34)                 
...                                                                                                                                   ...                 
/home/workspace/src/porting_layer/src/plat_path.c find_path_by_files 191   (/home/workspace/src/porting_layer/src/plat_path.c, find_path_by_files, 191)   
                                                                     192   (/home/workspace/src/porting_layer/src/plat_path.c, find_path_by_files, 192)   
                                                                     193   (/home/workspace/src/porting_layer/src/plat_path.c, find_path_by_files, 193)   
                                                                     194   (/home/workspace/src/porting_layer/src/plat_path.c, find_path_by_files, 194)   
                                                                     197   (/home/workspace/src/porting_layer/src/plat_path.c, find_path_by_files, 197)   

                                                                           e_p  \
file                                              function           line        
/home/workspace/src/common/chewing-utf8-util.c    ueStrLen           28    13    
                                                                     30    13    
                                                                     31    13    
                                                                     33    13    
                                                                     34    13    
...                                                                        ..    
/home/workspace/src/porting_layer/src/plat_path.c find_path_by_files 191   12    
                                                                     192   12    
                                                                     193   12    
                                                                     194   12    
                                                                     197   2     

                                                                           e_f  \
file                                              function           line        
/home/workspace/src/common/chewing-utf8-util.c    ueStrLen           28    1     
                                                                     30    1     
                                                                     31    1     
                                                                     33    1     
                                                                     34    1     
...                                                                       ..     
/home/workspace/src/porting_layer/src/plat_path.c find_path_by_files 191   1     
                                                                     192   1     
                                                                     193   1     
                                                                     194   1     
                                                                     197   0    

# Compute suspiciousness scores

In [6]:
def ochiai(e_f, e_p, n_f, n_p):
    return e_f / np.sqrt(e_p + e_f) / np.sqrt(e_f + n_f) if e_f else 0.0


def op2(e_f, e_p, n_f, n_p):
    return e_f - e_p / (e_p + n_p + 1)

spectra["Sop2"] = spectra.apply(
    lambda row: op2(row.e_f, row.e_p, row.n_f, row.n_p), axis=1
)
spectra["Sochiai"] = spectra.apply(
    lambda row: ochiai(row.e_f, row.e_p, row.n_f, row.n_p), axis=1
)
spectra["Rop2"] = spectra.Sop2.rank(ascending=False, method="max")
spectra["Rochiai"] = spectra.Sochiai.rank(ascending=False, method="max")


# Line level SBFL

In [7]:
for fault_meth in fault_meth2idx:
    fault_idxs_of_meth = fault_meth2idx[fault_meth]
    print("fault meth:", fault_meth)
    for fault_idx in fault_idxs_of_meth:
        print("fault idx:", tuple(fault_idx))
    print("SBFL score of faulty method:")
    display(spectra.loc[fault_idxs_of_meth].reset_index(drop=True))

print("Top 5 SBFL scores:")
print("Op2:")
display(
    spectra.sort_values("Sop2", ascending=False).head(5).reset_index(drop=True)
)
print("Ochiai:")
display(
    spectra.sort_values("Sochiai", ascending=False)
    .head(5)
    .reset_index(drop=True)
)


fault meth: ('/home/workspace/src/chewingutil.c', 'CallPhrasing')
fault idx: ('/home/workspace/src/chewingutil.c', 'CallPhrasing', 731)
fault idx: ('/home/workspace/src/chewingutil.c', 'CallPhrasing', 732)
SBFL score of faulty method:


,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai
0,"(/home/workspace/src/chewingutil.c, CallPhrasing, 731)",7,1,10,0,0.611111,0.353553,2060.0,2060.0
1,"(/home/workspace/src/chewingutil.c, CallPhrasing, 732)",7,1,10,0,0.611111,0.353553,2060.0,2060.0


Top 5 SBFL scores:
Op2:


,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai
0,"(/home/workspace/src/pinyin.c, PinyinToBopomofo, 151)",0,1,17,0,1.0,1.0,458.0,458.0
1,"(/home/workspace/src/bopomofo.c, PinYinInput, 554)",0,1,17,0,1.0,1.0,458.0,458.0
2,"(/home/workspace/src/bopomofo.c, PinYinInput, 534)",0,1,17,0,1.0,1.0,458.0,458.0
3,"(/home/workspace/src/bopomofo.c, PinYinInput, 535)",0,1,17,0,1.0,1.0,458.0,458.0
4,"(/home/workspace/src/bopomofo.c, PinYinInput, 540)",0,1,17,0,1.0,1.0,458.0,458.0


Ochiai:


,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai
0,"(/home/workspace/src/pinyin.c, PinyinToBopomofo, 151)",0,1,17,0,1.0,1.0,458.0,458.0
1,"(/home/workspace/src/bopomofo.c, PinYinInput, 554)",0,1,17,0,1.0,1.0,458.0,458.0
2,"(/home/workspace/src/bopomofo.c, PinYinInput, 534)",0,1,17,0,1.0,1.0,458.0,458.0
3,"(/home/workspace/src/bopomofo.c, PinYinInput, 535)",0,1,17,0,1.0,1.0,458.0,458.0
4,"(/home/workspace/src/bopomofo.c, PinYinInput, 540)",0,1,17,0,1.0,1.0,458.0,458.0


# Method level SBFL

In [8]:
spectra["file"] = spectra.index.map(lambda x: x[0])
spectra["func"] = spectra.index.map(lambda x: x[1])
spectra["line"] = spectra.index.map(lambda x: x[2])
# (file, func) is meth
spectra["meth"] = spectra.apply(lambda row: (row.file, row.func), axis=1)
spectra.reset_index(drop=True, inplace=True)
display(spectra)

,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai,file,func,meth
0,28,13,1,4,0,0.277778,0.267261,2809.0,2809.0,/home/workspace/src/common/chewing-utf8-util.c,ueStrLen,"(/home/workspace/src/common/chewing-utf8-util.c, ueStrLen)"
1,30,13,1,4,0,0.277778,0.267261,2809.0,2809.0,/home/workspace/src/common/chewing-utf8-util.c,ueStrLen,"(/home/workspace/src/common/chewing-utf8-util.c, ueStrLen)"
2,31,13,1,4,0,0.277778,0.267261,2809.0,2809.0,/home/workspace/src/common/chewing-utf8-util.c,ueStrLen,"(/home/workspace/src/common/chewing-utf8-util.c, ueStrLen)"
3,33,13,1,4,0,0.277778,0.267261,2809.0,2809.0,/home/workspace/src/common/chewing-utf8-util.c,ueStrLen,"(/home/workspace/src/common/chewing-utf8-util.c, ueStrLen)"
4,34,13,1,4,0,0.277778,0.267261,2809.0,2809.0,/home/workspace/src/common/chewing-utf8-util.c,ueStrLen,"(/home/workspace/src/common/chewing-utf8-util.c, ueStrLen)"
...,...,...,...,...,...,...,...,...,...,...,...,...
3519,191,12,1,5,0,0.333333,0.277350,2787.0,2787.0,/home/workspace/src/porting_layer/src/plat_path.c,find_path_by_files,"(/home/workspace/src/porting_layer/src/plat_path.c, find_path_by_files)"
3520,192,12,1,5,0,0.333333,0.277350,2787.0,2787.0,/home/workspace/src/porting_layer/src/plat_path.c,find_path_by_files,"(/home/workspace/src/porting_layer/src/plat_path.c, find_path_by_files)"
3521,193,12,1,5,0,0.333333,0.277350,2787.0,2787.0,/home/workspace/src/porting_layer/src/plat_path.c,find_path_by_files,"(/home/workspace/src/porting_layer/src/plat_path.c, find_path_by_files)"
3522,194,12,1,5,0,0.333333,0.277350,2787.0,2787.0,/home/workspace/src/porting_layer/src/plat_path.c,find_path_by_files,"(/home/workspace/src/porting_layer/src/plat_path.c, find_path_by_files)"


## Max aggregation

In [9]:
print("[Op2]")
spectra_max_op2 = spectra.copy().sort_values(
    ["Sop2", "meth", "line"], ascending=False
)
spectra_max_op2.drop_duplicates(subset="meth", keep="first", inplace=True)
spectra_max_op2 = spectra_max_op2[
    ["meth", "line", "e_p", "e_f", "n_p", "n_f", "Sop2"]
]
spectra_max_op2["Rop2"] = spectra_max_op2.Sop2.rank(
    ascending=False, method="max"
)
print("Faulty method:")
display(
    spectra_max_op2.loc[spectra_max_op2.meth.isin(fault_meths)].reset_index(
        drop=True
    )
)
print("Top 20 methods:")
display(spectra_max_op2.head(20).reset_index(drop=True))
print()
print("[Ochiai]")
spectra_max_ochiai = spectra.copy().sort_values(
    ["Sochiai", "meth", "line"], ascending=False
)
spectra_max_ochiai.drop_duplicates(subset="meth", keep="first", inplace=True)
spectra_max_ochiai = spectra_max_ochiai[
    ["meth", "line", "e_p", "e_f", "n_p", "n_f", "Sochiai"]
]
spectra_max_ochiai["Rochiai"] = spectra_max_ochiai.Sochiai.rank(
    ascending=False, method="max"
)
print("Faulty method:")
display(
    spectra_max_ochiai.loc[
        spectra_max_ochiai.meth.isin(fault_meths)
    ].reset_index(drop=True)
)
print("Top 20 methods:")
display(spectra_max_ochiai.head(20).reset_index(drop=True))


[Op2]
Faulty method:


,meth,line,e_p,e_f,n_p,n_f,Sop2,Rop2
0,"(/home/workspace/src/chewingutil.c, CallPhrasing)",739,1,1,16,0,0.944444,89.0


Top 20 methods:


,meth,line,e_p,e_f,n_p,n_f,Sop2,Rop2
0,"(/home/workspace/src/tree.c, TreeFindPhrase)",270,0,1,17,0,1.0,52.0
1,"(/home/workspace/src/tree.c, SortListByScore)",666,0,1,17,0,1.0,52.0
2,"(/home/workspace/src/tree.c, SortByIncreaseEnd)",851,0,1,17,0,1.0,52.0
3,"(/home/workspace/src/tree.c, SaveRecord)",697,0,1,17,0,1.0,52.0
4,"(/home/workspace/src/tree.c, RecursiveSave)",727,0,1,17,0,1.0,52.0
5,"(/home/workspace/src/tree.c, PhraseIntervalIntersect)",76,0,1,17,0,1.0,52.0
6,"(/home/workspace/src/tree.c, NextCut)",839,0,1,17,0,1.0,52.0
7,"(/home/workspace/src/tree.c, IsRecContain)",642,0,1,17,0,1.0,52.0
8,"(/home/workspace/src/tree.c, CountMatchCnnct)",782,0,1,17,0,1.0,52.0
9,"(/home/workspace/src/tree.c, CompRecord)",425,0,1,17,0,1.0,52.0



[Ochiai]
Faulty method:


,meth,line,e_p,e_f,n_p,n_f,Sochiai,Rochiai
0,"(/home/workspace/src/chewingutil.c, CallPhrasing)",739,1,1,16,0,0.707107,89.0


Top 20 methods:


,meth,line,e_p,e_f,n_p,n_f,Sochiai,Rochiai
0,"(/home/workspace/src/tree.c, TreeFindPhrase)",270,0,1,17,0,1.0,52.0
1,"(/home/workspace/src/tree.c, SortListByScore)",666,0,1,17,0,1.0,52.0
2,"(/home/workspace/src/tree.c, SortByIncreaseEnd)",851,0,1,17,0,1.0,52.0
3,"(/home/workspace/src/tree.c, SaveRecord)",697,0,1,17,0,1.0,52.0
4,"(/home/workspace/src/tree.c, RecursiveSave)",727,0,1,17,0,1.0,52.0
5,"(/home/workspace/src/tree.c, PhraseIntervalIntersect)",76,0,1,17,0,1.0,52.0
6,"(/home/workspace/src/tree.c, NextCut)",839,0,1,17,0,1.0,52.0
7,"(/home/workspace/src/tree.c, IsRecContain)",642,0,1,17,0,1.0,52.0
8,"(/home/workspace/src/tree.c, CountMatchCnnct)",782,0,1,17,0,1.0,52.0
9,"(/home/workspace/src/tree.c, CompRecord)",425,0,1,17,0,1.0,52.0


## Line2Method voting (weight: sbfl score / dense rank)

In [10]:
print("[Op2]")
ballot_op2 = {}
spectra_max_op2 = spectra.copy().sort_values(
    ["Sop2", "meth", "line"], ascending=False
)
spectra_max_op2["Rop2D"] = spectra_max_op2.Sop2.rank(
    ascending=False, method="dense"
)
spectra_max_op2["vote"] = spectra_max_op2.Sop2 / spectra_max_op2.Rop2D
for row in spectra_max_op2.itertuples():
    if row.meth not in ballot_op2:
        ballot_op2[row.meth] = row.vote
    else:
        ballot_op2[row.meth] += row.vote
L2M_op2 = pd.DataFrame(
    {"meth": list(ballot_op2.keys()), "vote": list(ballot_op2.values())}
)
L2M_op2["rank"] = L2M_op2.vote.rank(ascending=False, method="max")
L2M_op2.sort_values("rank", inplace=True)
L2M_op2.reset_index(drop=True, inplace=True)
for fault_meth in fault_meths:
    print("Faulty method:")
    display(L2M_op2.loc[L2M_op2.meth == fault_meth].reset_index(drop=True))
    print("Voters:")
    display(
        spectra_max_op2.loc[spectra_max_op2.meth == fault_meth].reset_index(
            drop=True
        )
    )
print("Top 5 methods:")
display(L2M_op2.head(5).reset_index(drop=True))
for idx, top_meth in enumerate(L2M_op2.head(5).meth.tolist(), 1):
    print(f"Top {idx}:")
    display(L2M_op2.loc[L2M_op2.meth == top_meth].reset_index(drop=True))
    print("Voters:")
    display(
        spectra_max_op2.loc[spectra_max_op2.meth == top_meth].reset_index(
            drop=True
        )
    )
print()
print("[Ochiai]")
ballot_ochiai = {}
spectra_max_ochiai = spectra.copy().sort_values(
    ["Sochiai", "meth", "line"], ascending=False
)
spectra_max_ochiai["RochiaiD"] = spectra_max_ochiai.Sochiai.rank(
    ascending=False, method="dense"
)
spectra_max_ochiai["vote"] = (
    spectra_max_ochiai.Sochiai / spectra_max_ochiai.RochiaiD
)
for row in spectra_max_ochiai.itertuples():
    if row.meth not in ballot_ochiai:
        ballot_ochiai[row.meth] = row.vote
    else:
        ballot_ochiai[row.meth] += row.vote
L2M_ochiai = pd.DataFrame(
    {"meth": list(ballot_ochiai.keys()), "vote": list(ballot_ochiai.values())}
)
L2M_ochiai["rank"] = L2M_ochiai.vote.rank(ascending=False, method="max")
L2M_ochiai.sort_values("rank", inplace=True)
L2M_ochiai.reset_index(drop=True, inplace=True)
for fault_meth in fault_meths:
    print("Faulty method:")
    display(
        L2M_ochiai.loc[L2M_ochiai.meth == fault_meth].reset_index(drop=True)
    )
    print("Voters:")
    display(
        spectra_max_ochiai.loc[
            spectra_max_ochiai.meth == fault_meth
        ].reset_index(drop=True)
    )
print("Top 5 methods:")
display(L2M_ochiai.head(5).reset_index(drop=True))
for idx, top_meth in enumerate(L2M_ochiai.head(5).meth.tolist(), 1):
    print(f"Top {idx}:")
    display(L2M_ochiai.loc[L2M_ochiai.meth == top_meth].reset_index(drop=True))
    print("Voters:")
    display(
        spectra_max_ochiai.loc[spectra_max_ochiai.meth == top_meth].reset_index(
            drop=True
        )
    )


[Op2]
Faulty method:


,meth,vote,rank
0,"(/home/workspace/src/chewingutil.c, CallPhrasing)",1.151852,129.0


Voters:


,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai,file,func,meth,Rop2D,vote
0,739,1,1,16,0,0.944444,0.707107,849.0,849.0,/home/workspace/src/chewingutil.c,CallPhrasing,"(/home/workspace/src/chewingutil.c, CallPhrasing)",2.0,0.472222
1,738,7,1,10,0,0.611111,0.353553,2060.0,2060.0,/home/workspace/src/chewingutil.c,CallPhrasing,"(/home/workspace/src/chewingutil.c, CallPhrasing)",8.0,0.076389
2,732,7,1,10,0,0.611111,0.353553,2060.0,2060.0,/home/workspace/src/chewingutil.c,CallPhrasing,"(/home/workspace/src/chewingutil.c, CallPhrasing)",8.0,0.076389
3,731,7,1,10,0,0.611111,0.353553,2060.0,2060.0,/home/workspace/src/chewingutil.c,CallPhrasing,"(/home/workspace/src/chewingutil.c, CallPhrasing)",8.0,0.076389
4,729,7,1,10,0,0.611111,0.353553,2060.0,2060.0,/home/workspace/src/chewingutil.c,CallPhrasing,"(/home/workspace/src/chewingutil.c, CallPhrasing)",8.0,0.076389
5,728,9,1,8,0,0.500000,0.316228,2145.0,2145.0,/home/workspace/src/chewingutil.c,CallPhrasing,"(/home/workspace/src/chewingutil.c, CallPhrasing)",10.0,0.050000
6,751,11,1,6,0,0.388889,0.288675,2753.0,2753.0,/home/workspace/src/chewingutil.c,CallPhrasing,"(/home/workspace/src/chewingutil.c, CallPhrasing)",12.0,0.032407
7,749,11,1,6,0,0.388889,0.288675,2753.0,2753.0,/home/workspace/src/chewingutil.c,CallPhrasing,"(/home/workspace/src/chewingutil.c, CallPhrasing)",12.0,0.032407
8,746,11,1,6,0,0.388889,0.288675,2753.0,2753.0,/home/workspace/src/chewingutil.c,CallPhrasing,"(/home/workspace/src/chewingutil.c, CallPhrasing)",12.0,0.032407
9,743,11,1,6,0,0.388889,0.288675,2753.0,2753.0,/home/workspace/src/chewingutil.c,CallPhrasing,"(/home/workspace/src/chewingutil.c, CallPhrasing)",12.0,0.032407


Top 5 methods:


...

,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai,file,func,meth,RochiaiD,vote
0,269,0,1,17,0,1.000000,1.000000,458.0,458.0,/home/workspace/src/bopomofo.c,HsuPhoInput,"(/home/workspace/src/bopomofo.c, HsuPhoInput)",1.0,1.000000
1,251,0,1,17,0,1.000000,1.000000,458.0,458.0,/home/workspace/src/bopomofo.c,HsuPhoInput,"(/home/workspace/src/bopomofo.c, HsuPhoInput)",1.0,1.000000
2,248,0,1,17,0,1.000000,1.000000,458.0,458.0,/home/workspace/src/bopomofo.c,HsuPhoInput,"(/home/workspace/src/bopomofo.c, HsuPhoInput)",1.0,1.000000
3,247,0,1,17,0,1.000000,1.000000,458.0,458.0,/home/workspace/src/bopomofo.c,HsuPhoInput,"(/home/workspace/src/bopomofo.c, HsuPhoInput)",1.0,1.000000
4,244,0,1,17,0,1.000000,1.000000,458.0,458.0,/home/workspace/src/bopomofo.c,HsuPhoInput,"(/home/workspace/src/bopomofo.c, HsuPhoInput)",1.0,1.000000
5,242,0,1,17,0,1.000000,1.000000,458.0,458.0,/home/workspace/src/bopomofo.c,HsuPhoInput,"(/home/workspace/src/bopomofo.c, HsuPhoInput)",1.0,1.000000
6,232,0,1,17,0,1.000000,1.000000,458.0,458.0,/home/workspace/src/bopomofo.c,HsuPhoInput,"(/home/workspace/src/bopomofo.c, HsuPhoInput)",1.0,1.000000
7,226,0,1,17,0,1.000000,1.000000,458.0,458.0,/home/workspace/src/bopomofo.c,HsuPhoInput,"(/home/workspace/src/bopomofo.c, HsuPhoInput)",1.0,1.000000
8,224,0,1,17,0,1.000000,1.000000,458.0,458.0,/home/workspace/src/bopomofo.c,HsuPhoInput,"(/home/workspace/src/bopomofo.c, HsuPhoInput)",1.0,1.000000
9,220,0,1,17,0,1.000000,1.000000,458.0,458.0,/home/workspace/src/bopomofo.c,HsuPhoInput,"(/home/workspace/src/bopomofo.c, HsuPhoInput)",1.0,1.000000


Top 5:


,meth,vote,rank
0,"(/home/workspace/src/bopomofo.c, ET26PhoInput)",35.303301,5.0


Voters:


,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai,file,func,meth,RochiaiD,vote
0,365,0,1,17,0,1.000000,1.000000,458.0,458.0,/home/workspace/src/bopomofo.c,ET26PhoInput,"(/home/workspace/src/bopomofo.c, ET26PhoInput)",1.0,1.000000
1,358,0,1,17,0,1.000000,1.000000,458.0,458.0,/home/workspace/src/bopomofo.c,ET26PhoInput,"(/home/workspace/src/bopomofo.c, ET26PhoInput)",1.0,1.000000
2,353,0,1,17,0,1.000000,1.000000,458.0,458.0,/home/workspace/src/bopomofo.c,ET26PhoInput,"(/home/workspace/src/bopomofo.c, ET26PhoInput)",1.0,1.000000
3,352,0,1,17,0,1.000000,1.000000,458.0,458.0,/home/workspace/src/bopomofo.c,ET26PhoInput,"(/home/workspace/src/bopomofo.c, ET26PhoInput)",1.0,1.000000
4,348,0,1,17,0,1.000000,1.000000,458.0,458.0,/home/workspace/src/bopomofo.c,ET26PhoInput,"(/home/workspace/src/bopomofo.c, ET26PhoInput)",1.0,1.000000
5,344,0,1,17,0,1.000000,1.000000,458.0,458.0,/home/workspace/src/bopomofo.c,ET26PhoInput,"(/home/workspace/src/bopomofo.c, ET26PhoInput)",1.0,1.000000
6,333,0,1,17,0,1.000000,1.000000,458.0,458.0,/home/workspace/src/bopomofo.c,ET26PhoInput,"(/home/workspace/src/bopomofo.c, ET26PhoInput)",1.0,1.000000
7,327,0,1,17,0,1.000000,1.000000,458.0,458.0,/home/workspace/src/bopomofo.c,ET26PhoInput,"(/home/workspace/src/bopomofo.c, ET26PhoInput)",1.0,1.000000
8,326,0,1,17,0,1.000000,1.000000,458.0,458.0,/home/workspace/src/bopomofo.c,ET26PhoInput,"(/home/workspace/src/bopomofo.c, ET26PhoInput)",1.0,1.000000
9,323,0,1,17,0,1.000000,1.000000,458.0,458.0,/home/workspace/src/bopomofo.c,ET26PhoInput,"(/home/workspace/src/bopomofo.c, ET26PhoInput)",1.0,1.000000


# 감상

단순히 passing test에 의해서 한 번도 실행되지 않고 failing test에 의해서 실행되는 method가 너무 많다 (sbfl max agg top max rank: 52)